In [1]:
import pandas as pd
import numpy as np
from PIL import Image

### 实现image与pandas矩阵互转

In [2]:
## 这个函数将图片转为pandas灰度矩阵
def grayscale_to_matrix(image_path):
    # 打开图像
    image = Image.open(image_path)
    # 将图像转换为灰度图像
    grayscale_image = image.convert("L")
    # 转换为 numpy 数组
    image_np = np.array(grayscale_image)
    #转换为pandas矩阵
    image_df=pd.DataFrame(image_np)
    return image_np,image_df

##这个函数将矩阵转换为图片，传入一个pandas矩阵
def matrix_to_image(matrix,name):
    # 将 Pandas DataFrame 转换为 NumPy 数组
    image_array = matrix.to_numpy(dtype='uint8')
    # 创建 Image 对象
    image = Image.fromarray(image_array)
    # 保存图像为 BMP 文件
    image.save(name)
    return 

### 实现csv文件与pandas矩阵互转

In [3]:
def to_csvs(df):
    df.to_csv("out.csv",index=False, header=False)#不保存行索引和列索引
    
def csv_to_matrix(file_path):
    # 从CSV文件加载数据并转换为Pandas DataFrame
    matrix = pd.read_csv(file_path, header=None)#header=None是说不要将第一行转换为列名
    return matrix

### 将大图像切成若干图像块

In [4]:
# n1:图像块的高
# n2:图像块的宽
def split_matrix(matrix,n1,n2):
    # 计算矩阵的行数和列数
    rows, cols = matrix.shape
    # 创建一个空的字典来存储子矩阵
    sub_matrices = {}
    
    # 将矩阵分割为若干个 n1*n2 的子矩阵
    index = 1
    for i in range(0, rows, n1):
        for j in range(0, cols, n2):
            sub_matrix = matrix.iloc[i:i+n1, j:j+n2]
            sub_matrices[index] = sub_matrix
            index += 1
    return sub_matrices

### 将若干图像块矩阵拼成一个大图像矩阵

In [5]:
# 定义最开始的大图像矩阵被分割成“m1*m2”个小矩阵
# m1是指大图像矩阵一行上被分割为多少个小矩阵,取决于图像块block的大小

def combine_dataframes(dictionary,m1):
    # 按照键的大小顺序对字典中的 DataFrame 进行排序
    sorted_dfs = sorted(dictionary.items(), key=lambda x: x[0])
    
    # 将排序后的 DataFrame 以每 m1 个一组的方式拼接成一行
    rows = [pd.concat([df for _, df in sorted_dfs[i:i+m1]], axis=1) for i in range(0, len(sorted_dfs), m1)]
    
    # 将拼接后的每行 DataFrame 组合成一个大的 DataFrame
    result_df = pd.concat(rows, axis=0)
    
    return result_df


### 将图像块的灰度矩阵转化为灰度值序列

In [6]:
def matrix_to_dict(matrix):
    # 将 Pandas DataFrame 转换为 NumPy 数组
    array = matrix.to_numpy()
    # 创建一个空的字典
    result_dict = {}    #result_dict记录的是原始顺序的灰度矩阵信息，以“原始序号：灰度值”的形式存储
    # 获取矩阵的行数和列数
    rows, cols = array.shape
    # 遍历矩阵中的每个元素，并按顺序编号存储到字典中
    index = 1
    for i in range(rows):
        for j in range(cols):
            result_dict[index] = array[i][j]
            index += 1
    #sorted_dict记录的是排序顺序的灰度矩阵信息，以“原始序号：灰度值”的形式存储
    sorted_list = sorted(result_dict.items(), key=lambda x: x[1], reverse=True)
    return result_dict,sorted_list

### 将灰度值序列转化为图像块灰度矩阵

In [7]:
# n1:图像块的高
# n2:图像块的宽
def list_to_dataframe(data,n1,n2):
    # 生成一个2x2的空DataFrame
    matrix = pd.DataFrame(index=range(n1), columns=range(n2))
    
    # 按照元组的第一个元素排序
    sorted_data = sorted(data, key=lambda x: x[0])
    
    # 逐个填充到DataFrame中
    for idx, (i, j) in enumerate(sorted_data):
        matrix.iloc[idx // n2, idx % n2] = j
    
    return matrix

### 计算e_max

In [8]:
def calculate_e_max(block_matrix):#传入一个图像块的灰度矩阵
    #将灰度矩阵的像素点按灰度值大小排序
    #sorted_list是这样的：[(2, 199), (4, 196), (1, 65), (3, 54)]
    #以(2, 199)为例，2是像素点排序前的的原始编号，199是像素点的灰度值
    
    _,sorted_list=matrix_to_dict(block_matrix) 
    k=0
    temp=sorted_list[0][1] #存储排在最前面的像素点的灰度值
    for i in sorted_list:
        if i[1]==temp:
            k=k+1
        else :
            break
    #if k==len(sorted_list):#说明这个block块的所有灰度值都相等，另其e_max=0         
    if k ==len(sorted_list):
        e_max=0
        return e_max,k
    e_max=int(sorted_list[0][1])-int(sorted_list[k][1])
    return e_max,k
    

### 二进制序列与文字序列互转

In [9]:
def binary_to_text(binary_sequence):
    # 将二进制序列按照每8位拆分为一个字节
    bytes_list = [binary_sequence[i:i+8] for i in range(0, len(binary_sequence), 8)]
    # 将每个字节转换为对应的 ASCII 字符
    text = ''.join(chr(int(byte, 2)) for byte in bytes_list)
    return text

def text_to_binary(text):
    # 将字符串转换为 ASCII 编码的字节序列
    bytes_text = text.encode('ascii')
    # 将字节序列转换为二进制字符串
    binary_text = ''.join(format(byte, '08b') for byte in bytes_text)
    bit_list = [int(bit) for bit in binary_text]
    print("嵌入信息大小为："+str(len(bit_list))+" bits")
    return binary_text,bit_list

### 找到一个DataFrame中的最大值以及其对应的逻辑索引

In [10]:
#找TOP1的像素点编号，k=1，找TOP2的像素点编号，k=2
#n2是图像块的宽度
def find_max_value_and_position(dataframe,k,n2):
    #sorted_list是这样的：[(2, 199), (4, 196), (1, 65), (3, 54)]
    _,sort_list=matrix_to_dict(j)
    #讨论想找最大的元素但是前两个最大元素的值都相同该怎么办:两个序号都返回
    #row1,col1是大序号的元素的索引，row2,col2是小序号的元素的索引
    if k==1 and sort_list[0][1]==sort_list[1][1]:
        number1=max(sort_list[0][0],sort_list[1][0])
        row1=(number1-1)//n2
        col1=(number1-1)%n2
        number2=min(sort_list[0][0],sort_list[1][0])
        row2=(number2-1)//n2
        col2=(number2-1)%n2
        return row1,col1,row2,col2
    number=sort_list[k-1][0]
    row=(number-1)//n2
    col=(number-1)%n2
    return row, col

In [11]:
def num_to_loc(number,n2): #根据像素点的序号，返回其在灰度矩阵的物理坐标
    #n2是图像块block的宽，即列数
    row=(number-1)//n2
    col=(number-1)%n2
    return row,col

In [12]:
#检查嵌入到这个图像块中的序列类型
def check_list_type(lst):
    all_zeros = all(x == 0 for x in lst)
    all_ones = all(x == 1 for x in lst)
    mixed = not all_zeros and not all_ones
    
    if all_zeros:
        return 0
    elif all_ones:
        return 1
    elif mixed:
        return 10

# 主函数

### 加密部分

In [13]:
image_np,image_df=grayscale_to_matrix("Lena.bmp")

In [14]:
#打印载入的灰度矩阵
image_df

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,162,162,162,161,162,157,163,161,165,161,...,120,124,138,163,168,170,171,170,155,128
1,162,162,162,161,162,157,163,161,165,161,...,120,124,138,163,168,170,171,170,155,128
2,162,162,162,161,162,157,163,161,165,161,...,120,124,138,163,168,170,171,170,155,128
3,162,162,162,161,162,157,163,161,165,161,...,120,124,138,163,168,170,171,170,155,128
4,162,162,162,161,162,157,163,161,165,161,...,120,124,138,163,168,170,171,170,155,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,54,54,48,47,53,53,50,55,56,60,...,55,64,67,78,83,89,90,93,94,90
508,46,46,50,50,52,49,50,51,58,55,...,62,71,75,81,89,93,100,102,98,96
509,43,43,50,47,54,47,50,49,60,52,...,70,69,79,81,92,97,100,104,100,98
510,44,44,55,51,54,47,50,51,58,51,...,76,83,94,92,97,105,100,104,105,108


In [15]:
#图像切割成若干1*2的图像块
# n1:图像块的高
# n2:图像块的宽
n1=2
n2=2
sub_matrices=split_matrix(image_df,n1,n2)
#打印切割的图像块以检查分割情况
sub_matrices[1]

,0,1
0,162,162
1,162,162


In [16]:
#计算每个图像块的e_max
sub_blocks={}#这个与sub_matrices的区别在于它记录了e_max的信息
for (i,j) in zip(sub_matrices.keys(),sub_matrices.values()):
    e_max,k=calculate_e_max(j)
    sub_blocks[(i,e_max,k)]=j

In [17]:
#计算这张图片的最大嵌入信息量
max_count=0
for i in sub_blocks.keys():
    if  i[1]==1:
        max_count=max_count+i[2]
print("最大嵌入信息量为："+str(max_count)+" bit")

最大嵌入信息量为：23822 bit


In [18]:
##开始信息嵌入
##准备加密信息
secret_text="Wish you a good day,my best friend!"
_ ,data_1=text_to_binary(secret_text) #将加密信息转化为二进制list
data_2=[0] * ( max_count-len(data_1) )  #因为一般情况下最大可嵌入数据大于待嵌入数据，因此还要设定一下填充序列
data=data_1+data_2

嵌入信息大小为：280 bits


In [19]:
cnt=0
for i,j in zip(sub_blocks.keys(),sub_blocks.values()):
    #把每一个block的灰度矩阵转化成类似于这样的序列：[(2, 199), (4, 196), (1, 65), (3, 54)]
    _,sort_list=matrix_to_dict(j) 
    if i[1]==1: # i[0]是图像块的序号, i[1]是e_max, i[2]是k值
        #首先将前k个相等的像素点取出
        lists=[sort_list[p] for p in range(0,i[2])]
        #将前面k个灰度值相等的像素点单独取出,并按照其序号角标从小到大排序，以确定嵌入数据的顺序
        sorted_data=sorted(lists,key=lambda x :x[0])
        #检查嵌入序列的类型已选择不同的嵌入方式
        check=check_list_type(data[cnt:cnt+len(sorted_data)])
        #检查嵌入信息，如果嵌入这个block的信息是全0那么全部像素点加1，如果block的信息是全1，那么全部像素点加2
        if check==0:
            for r in sorted_data :
                row,col=num_to_loc(r[0],n2)
                sub_blocks[i].iloc[row,col]=r[1]+1
                cnt+=1
        if check==1:
            for r in sorted_data :
                row,col=num_to_loc(r[0],n2)
                sub_blocks[i].iloc[row,col]=r[1]+2
                cnt+=1
        if check==10:
            for r in sorted_data :
                row,col=num_to_loc(r[0],n2)
                sub_blocks[i].iloc[row,col]=r[1]+data[cnt]
                cnt+=1
                
    elif i[1] >1:
        #对于e_max原本就大于1的block,将其 e_max=e_max+2
        for m in range(0,i[2]):
            row,col=num_to_loc(sort_list[m][0],n2)
            sub_blocks[i].iloc[row,col]=sort_list[m][1]+2


In [20]:
sub_blocks_final={}
for i,j in zip(sub_blocks.keys(),sub_blocks.values()):
    sub_blocks_final[i[0]]=j   
    
##将修改后的图像块拼成一个大的
modified_matrix=combine_dataframes(sub_blocks_final,256) #512*512的图像被2*2的block分割，一行可以分出256个block
#将加密图像还原
matrix_to_image(modified_matrix,"secret_image.bmp")

### 解密部分

In [21]:
image_np,image_df=grayscale_to_matrix("secret_image.bmp")

In [22]:
#图像切割成若干1*2的图像块
# n1:图像块的高
# n2:图像块的宽
n1=2
n2=2
sub_matrices=split_matrix(image_df,n1,n2)
#打印切割的图像块以检查分割情况
sub_matrices[1]

,0,1
0,162,162
1,162,162


In [23]:
#计算每个图像块的e_max，这里是修改后的e_max
sub_blocks={}#这个与sub_matrices的区别在于它记录了e_max的信息
for (i,j) in zip(sub_matrices.keys(),sub_matrices.values()):
    e_max,k=calculate_e_max(j)
    sub_blocks[(i,e_max,k)]=j

In [24]:
#开始信息提取，并恢复原来的图像
massage_binary=""
for i,j in zip(sub_blocks.keys(),sub_blocks.values()):
    # i[0]是图像块的序号, i[1]是e_max, i[2]是k值
    _,sort_list=matrix_to_dict(j) #把灰度矩阵转化成类似于这样的序列[(2, 199), (4, 196), (1, 65), (3, 54)]
    if i[1]>3: 
        for m in range(0,i[2]):
            row,col=num_to_loc(sort_list[m][0],n2)
            sub_blocks[i].iloc[row,col]=sort_list[m][1]-2

  
    if i[1]==2:
        for m in range(0,i[2]):
            massage_binary=massage_binary+"0"
            row,col=num_to_loc(sort_list[m][0],n2)
            sub_blocks[i].iloc[row,col]=sort_list[m][1]-1
    if i[1]==3:
        for m in range(0,i[2]):
            massage_binary=massage_binary+"1"
            row,col=num_to_loc(sort_list[m][0],n2)
            sub_blocks[i].iloc[row,col]=sort_list[m][1]-2
        
    
    
    if i[1]==1:
        #先取出sort_list中前面k1个元素，这些元素嵌入了1
        #将sort_list前k1个元素删掉，再取出前k2个元素，这些元素嵌入了0
        #原始的k=k1+k2
        #计算k1_list
        list_k1=[]
        #k1已经在最开始计算出来了，就是i[2]
        for m in range(0,i[2]):
            list_k1.append(sort_list[m])
        #删掉前面的k1个元素
        sort_list=sort_list[i[2]:]
        
        #计算k2_list
        list_k2=[]
        k2=0
        temp=sort_list[0][1] #存储排在最前面的像素点的灰度值
        for g in sort_list:
            if g[1]==temp:
                k2=k2+1
            else :
                break     
        for m in range(0,k2):
            list_k2.append(sort_list[m])
        
        #信息提取
        
        list_final=list_k1 + list_k2
        big=0
        small=256
        for t in list_final:
            if t[1]>big:
                big=t[1]
            if t[1]<small:
                small=t[1]
                
        #sorted_data是这个样式的序列[(2, 199), (4, 196), (1, 65), (3, 54)]，将其按照内部元组的第一个元素的值排序
        sorted_data=sorted(list_final,key=lambda x :x[0])

        for t in sorted_data:
            if t[1]==big:
                massage_binary=massage_binary+"1"
            if t[1]==small:
                massage_binary=massage_binary+"0"
        #图像还原(因为list_k2中的像素点没有做改动，它嵌入的信息是0)
        for t in list_k1:
            row,col=num_to_loc(t[0],n2)
            sub_blocks[i].iloc[row,col]=t[1]-1

In [25]:
sub_blocks_final={}
for i,j in zip(sub_blocks.keys(),sub_blocks.values()):
    sub_blocks_final[i[0]]=j    

In [26]:
## 查看加密信息
message=binary_to_text(massage_binary)
print(message)

Wish you a good day,my best friend!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [27]:
##将更新后的图像块拼成一个大的
modified_matrix=combine_dataframes(sub_blocks_final,256) #512*512的图像被2*2的block分割，一行可以分出256个block
## 将还原的图像保存
matrix_to_image(modified_matrix,"Restored_image.bmp")

### PSNR计算

In [28]:
def calculate_psnr(original, reconstructed):
    # 计算均方误差
    mse = np.mean((original - reconstructed) ** 2)
    # 计算PSNR
    max_val = 255  # 8位灰度图像的最大像素值
    PSNR = 10 * np.log10((max_val ** 2) / mse)
    return PSNR

def calculate_mse(original, reconstructed):
    # 计算均方误差
    mse = np.mean((original - reconstructed) ** 2)
    return mse

original,_=grayscale_to_matrix('Lena.bmp')
modified,_=grayscale_to_matrix("secret_image.bmp")
restored,_=grayscale_to_matrix("Restored_image.bmp")

psnr_value = calculate_psnr(original, modified)
mse_value=calculate_mse(original,restored)

print("PSNR :", psnr_value)
print("MSE :", mse_value)

PSNR : 48.571762327736465
MSE : 0.0


### 草稿区

In [29]:
data = {
    '6': [191,191],
    '7': [189,192]
}
df_from_dict = pd.DataFrame(data)

In [30]:
q1=[1,2,3,4,5,6]
q1[1:]

[2, 3, 4, 5, 6]

In [31]:
[1,3,5]+[6,4,2]

[1, 3, 5, 6, 4, 2]

In [32]:
df_from_dict

,6,7
0,191,189
1,191,192


In [33]:
def calculate_e_max(block_matrix):#传入一个图像块的灰度矩阵
    #将灰度矩阵的像素点按灰度值大小排序
    #sorted_list是这样的：[(2, 199), (4, 196), (1, 65), (3, 54)]
    #以(2, 199)为例，2是像素点排序前的的原始编号，199是像素点的灰度值
    
    _,sorted_list=matrix_to_dict(block_matrix) 
    #print(sorted_list)
    k=0
    temp=sorted_list[0][1] #存储排在最前面的像素点的灰度值
    for i in sorted_list:
        if i[1]==temp:
            k=k+1
        else :
            break
            
    if k ==len(sorted_list):
        e_max=0
        return e_max,k
    e_max=int(sorted_list[0][1])-int(sorted_list[k][1])
    return e_max,k
    #if k==len(sorted_list):#说明这个block块的所有灰度值都相等，另其e_max=0
    #    e_max=0
    #else :
    #    e_max=int(sorted_list[0][1])-int(sorted_list[k][1])
   # return e_max,k

In [34]:
calculate_e_max(df_from_dict)

(1, 1)

In [35]:
for m in range(0,0):
    print(1)

In [36]:
c='0'+'1'
c

'01'